## VFE

Let $f=\{f^*,\mathrm{f}\}$ be the noise free latent variables of a $\mathcal{GP}$. We write $\mathrm{f}$ to be the latent variables where we have noisy observations $y$: $y_i = \mathrm{f}_i + \epsilon_i$. We want to approximate the true posterior: $q(f)\approx p(f \mid y)$. To do this we use the standard bound used in variational inference:

$$
\begin{aligned}
p(y ) &= \overbrace{\mathbb{E}_{q(f)}\left[\log \frac{p(y,f)}{q(f)} \right]}^{\mathcal{L}(q)} +\overbrace{\mathbb{E}_{q(f)}\left[\log \frac{p(f \mid y)}{q(f)}\right]}^{KL\left[q \mid\mid p\right]}\\
&\geq \mathcal{L}(q)
\end{aligned}
$$

Thus, maximizing $\mathcal{L}(q)$ is equivalent to minimizing the Kullback-Leiveck divergence between $q$ and the true posterior $p(f\mid y)$. 


Let $u$ be a subset of $f$: $u \subset f$: $f = \{f^*,\mathrm{f}, u\} = \{f_{\ne u}, u\}$. We can rewrite $\mathcal{L}$ as:

$$
\begin{aligned}
\mathcal{L}(q) &= \mathbb{E}_{q(f)}\left[\log \frac{p(y\mid f)p(f)}{q(f)} \right] \\
&= \mathbb{E}_{q(f)}\left[\log \frac{p(y\mid f)p(f_{\ne u} \mid u) p(u)}{q(f)} \right] \\
&= \mathbb{E}_{q(f)}\left[\log \frac{p(y\mid \mathrm{f})p(f^*, \mathrm{f} \mid u) p(u)}{q(f)} \right] 
\end{aligned}
$$

We can also rewrite the true posterior $p(f\mid y)$ using the subset $u$ of $f$:
$$
p(f\mid y) = p(f_{\ne u} \mid u,y)p(u \mid y)
$$

The Titsias approximation to the posterior chooses $q$ as:
$$
\begin{aligned}
q(f) &= \overbrace{p(f_{\ne u} \mid u)}^{\mathcal{N}(f_{\ne u} \mid K_{f_{\ne u}u}K_{uu}^{-1}u,\: K_{f_{\ne u}f_{\ne u}} - Q_{f_{\ne u}f_{\ne u}} )}q(u)\\
 &= p(f^*,\mathrm{f} \mid u) q(u)\\
 &= \mathcal{N}\left(\begin{pmatrix} \mathrm{f} \\ f^*  \end{pmatrix} \mid \begin{pmatrix} K_{\mathrm{f}u} \\ K_{*u}\end{pmatrix}K_{uu}^{-1}u,\: \begin{pmatrix} K_{\mathrm{f}\mathrm{f}} & K_{\mathrm{f}*} \\
 K_{*\mathrm{f}} & K_{**}\end{pmatrix} - \begin{pmatrix} Q_{\mathrm{f}\mathrm{f}} & Q_{\mathrm{f}*} \\
 Q_{*\mathrm{f}} & Q_{**}\end{pmatrix} \right) q(u)
\end{aligned}
$$
We see that **this approximation removes the dependency on the data ($y$) in the first term**. The second term $q(u)$ is let free. 

Substituting this $q$ in the bound $\mathcal{L}$ leads to:

$$
\begin{aligned}
\require{cancel}
\mathcal{L}(q) &= \mathbb{E}_{q(f)}\left[\log \frac{p(y\mid \mathrm{f})\cancel{p(f_{\ne u} \mid u)} p(u)}{\cancel{p(f_{\ne u} \mid u)}q(u)}\right] \\
&= \mathbb{E}_{q(f)}\left[\log p(y\mid \mathrm{f})\right] + \mathbb{E}_{p(f_{\ne u} \mid u)q(u)}\left[\log \frac{p(u)}{q(u)}\right] \\
&= \mathbb{E}_{q(f)}\left[\log p(y\mid \mathrm{f})\right] + \int \int \left[\log \frac{p(u)}{q(u)}\right] p(f_{\ne u} \mid u)q(u) df_{\ne u} du \\
&= \mathbb{E}_{q(f)}\left[\log p(y\mid \mathrm{f})\right] + \int \left[\log \frac{p(u)}{q(u)}\right] q(u) \overbrace{\left(\int p(f_{\ne u} \mid u) df_{\ne u} \right)}^{=1}  du \quad \text{Trick (1)}\\
&= \mathbb{E}_{q(f)}\left[\log p(y\mid \mathrm{f})\right] + KL\left[q(u)\mid\mid p(u)\right] \\
&= \mathbb{E}_{q(f)}\left[\log \prod_{i=1}^N p(y_i\mid \mathrm{f}_i)\right] + KL\left[q(u)\mid\mid p(u)\right] \quad \text{(step 6)} \\
&= \mathbb{E}_{q(\mathrm{f})}\left[\sum_{i=1}^N \log  p(y_i\mid \mathrm{f}_i)\right] + KL\left[q(u)\mid\mid p(u)\right]\\
&= \sum_{i=1}^N \mathbb{E}_{q(\mathrm{f})}\left[\log  p(y_i\mid \mathrm{f}_i)\right] + KL\left[q(u)\mid\mid p(u)\right] \\
&= \sum_{i=1}^N \mathbb{E}_{q(\mathrm{f})}\left[\frac{-1}{2}\log(2\pi\sigma^2) - \frac{1}{2\sigma^2}(y_i- f_i)^2\right] + KL\left[q(u)\mid\mid p(u)\right] \\
&= \sum_{i=1}^N \int \left[\frac{-1}{2}\log(2\pi\sigma^2) - \frac{1}{2\sigma^2}(y_i- f_i)^2\right]q(\mathrm{f}_i)\overbrace{\int q(f_1,..,f_{i-1},f_{i+1},..,f_N \mid \mathrm{f}_i) d(\mathrm{f}_1,..,\mathrm{f}_{i-1},\mathrm{f}_{i+1},..,\mathrm{f}_N)}^{=1}d\mathrm{f}_i + KL\left[q(u)\mid\mid p(u)\right] \\
&= \sum_{i=1}^N \mathbb{E}_{q(\mathrm{f}_i)}\left[\frac{-1}{2}\log(2\pi\sigma^2) - \frac{1}{2\sigma^2}(y_i- \mathrm{f}_i)^2\right]  + KL\left[q(u)\mid\mid p(u)\right]\\
&= \frac{-N}{2}\log(2\pi\sigma^2)+\sum_{i=1}^N \mathbb{E}_{q(\mathrm{f}_i)}\left[ - \frac{1}{2\sigma^2}(y_i- \mathrm{f}_i)^2\right]  + KL\left[q(u)\mid\mid p(u)\right] \\
&= \frac{-N}{2}\log(2\pi\sigma^2)+\sum_{i=1}^N \mathbb{E}_{q(u)}\left[\mathbb{E}_{q(\mathrm{f}_i \mid u)}\left[ - \frac{1}{2\sigma^2}(y_i- \mathrm{f}_i)^2\right]\right]  + KL\left[q(u)\mid\mid p(u)\right] \\
&= \frac{-N}{2}\log(2\pi\sigma^2)+\sum_{i=1}^N \mathbb{E}_{q(u)}\left[\mathbb{E}_{p(\mathrm{f}_i \mid u)}\left[ - \frac{1}{2\sigma^2}(y_i- \mathrm{f}_i)^2\right]\right]  + KL\left[q(u)\mid\mid p(u)\right] \\
&= \frac{-N}{2}\log(2\pi\sigma^2)+\sum_{i=1}^N \mathbb{E}_{q(u)}\left[\mathbb{E}_{p(\mathrm{f}_i \mid u)}\left[ - \frac{1}{2\sigma^2}(y_i^2+ \mathrm{f}_i^2 -2y_i \mathrm{f}_i)\right]\right]  + KL\left[q(u)\mid\mid p(u)\right] \\
&= \frac{-N}{2}\log(2\pi\sigma^2)+\sum_{i=1}^N \mathbb{E}_{q(u)}\left[- \frac{1}{2\sigma^2}\left(y_i^2+ \mathbb{E}_{p(\mathrm{f}_i \mid u)}[\mathrm{f}_i^2] -2y_i \mathbb{E}_{p(\mathrm{f}_i \mid u)}[\mathrm{f}_i]\right)\right]  + KL\left[q(u)\mid\mid p(u)\right] \quad \text{Using }p(\mathrm{f}_i \mid u ) = \mathcal{N}(K_{\mathrm{f}_iu}K_{uu}^{-1}u, K_{\mathrm{f}_i\mathrm{f}_i} - Q_{\mathrm{f}_i\mathrm{f}_i}) \\
&= \frac{-N}{2}\log(2\pi\sigma^2)+\sum_{i=1}^N \mathbb{E}_{q(u)}\left[- \frac{1}{2\sigma^2}\left(y_i^2+ K_{\mathrm{f}_i\mathrm{f}_i} - Q_{\mathrm{f}_i\mathrm{f}_i} + K_{\mathrm{f}_iu}K_{uu}^{-1}uu^tK_{uu}^{-1}K_{u\mathrm{f}_i} -2y_i K_{\mathrm{f}_iu}K_{uu}^{-1}u\right)\right]  + KL\left[q(u)\mid\mid p(u)\right]   \\
\end{aligned}
$$

If we assume $q(u) = \mathcal{N}(u \mid m,S)$

Alternative derivation from step (6):

$$
\begin{aligned}
\mathcal{L}(q)&= \mathbb{E}_{q(\mathrm{f})}\left[\log \prod_{i=1}^N p(y_i\mid \mathrm{f}_i)\right] + KL\left[q(u)\mid\mid p(u)\right] \\
&= \mathbb{E}_{p(\mathrm{f}\mid u)q(u)}\left[\log p(y \mid \mathrm{f})\right] + KL\left[q(u)\mid\mid p(u)\right] \\
&= \mathbb{E}_{q(u)}\left[\mathbb{E}_{p(\mathrm{f} \mid u)}\left[\log p(y \mid \mathrm{f})\right]\right] + KL\left[q(u)\mid\mid p(u)\right] \\
&= \mathbb{E}_{q(u)}\left[\mathbb{E}_{p(\mathrm{f} \mid u)}\left[\log p(y \mid \mathrm{f})\right]+\log \frac{ p(u)}{q(u)}\right]  \\
\end{aligned}
$$

This integral plays a central role in the derivation $\mathbb{E}_{p(\mathrm{f} \mid u)}\left[\log p(y \mid \mathrm{f})\right]$. Let's try to compute it:

$$
\begin{aligned}
\mathbb{E}_{p(\mathrm{f} \mid u)}\left[\log p(y \mid \mathrm{f})\right] &= \mathbb{E}_{p(\mathrm{f} \mid u)}\left[\log \mathcal{N}(y \mid \mathrm{f}, \sigma^2 I)\right] \\
&= \mathbb{E}_{p(\mathrm{f} \mid u)}\left[\log \left( \left(2\pi \sigma^{2N}\right)^{-1/2} \exp\left(\frac{-1}{2\sigma^{2}} \|y -\mathrm{f}\|^2\right)\right)\right] \\
&= \mathbb{E}_{p(\mathrm{f} \mid u)}\left[\log \left( \left(2\pi \sigma^{2N}\right)^{-1/2} \right) + \frac{-1}{2\sigma^{2}} \|y -\mathrm{f}\|^2\right] \\
&= \log \left( \left(2\pi \sigma^{2N}\right)^{-1/2} \right) + \mathbb{E}_{p(\mathrm{f} \mid u)}\left[\frac{-1}{2\sigma^{2}} \|y -\mathrm{f}\|^2\right] \\
&= \log \left( \left(2\pi \sigma^{2N}\right)^{-1/2} \right) + \frac{-1}{2\sigma^{2}}\mathbb{E}_{p(\mathrm{f} \mid u)}\left[ \left(\|y\|^2 + \|\mathrm{f}\|^2   -2 y^t \mathrm{f}\right)\right] \\
&= \log \left( \left(2\pi \sigma^{2N}\right)^{-1/2} \right) + \frac{-1}{2\sigma^{2}}\left( \|y\|^2 + \mathbb{E}_{p(\mathrm{f} \mid u)}\left[\|\mathrm{f}\|^2\right]   -2 y^t \mathbb{E}_{p(\mathrm{f} \mid u)}\left[\mathrm{f}\right]\right) \\
\end{aligned}
$$

Now the trick is to use that $p(\mathrm{f} \mid u) = \mathcal{N}(y \mid \mu_u, \mathrm{Cov}_u)$
$$
\begin{aligned}
\mathbb{E}_{p(\mathrm{f} \mid u)}\left[\log p(y \mid \mathrm{f})\right] &= \log \left( \left(2\pi \sigma^{2N}\right)^{-1/2} \right) + \frac{-1}{2\sigma^{2}}\left( y^t y + \mathbb{E}_{p(\mathrm{f} \mid u)}\left[\mathrm{f}^t \mathrm{f}\right]   -2 y^t \mu_u\right) \\
&= \log \left( \left(2\pi \sigma^{2N}\right)^{-1/2} \right) + \frac{-1}{2\sigma^{2}}\left( y^t y + \mathbb{E}_{p(\mathrm{f} \mid u)}\left[(\mathrm{f}-\mu_u)^t (\mathrm{f}-\mu_u) +2\mu_u^t f - \mu_u^t\mu_u\right]   -2 y^t \mu_u\right) \\
&= \log \left( \left(2\pi \sigma^{2N}\right)^{-1/2} \right) + \frac{-1}{2\sigma^{2}}\left( y^t y + \mathbb{E}_{p(\mathrm{f} \mid u)}\left[(\mathrm{f}-\mu_u)^t (\mathrm{f}-\mu_u) \right] +2\mu_u^t \mathbb{E}_{p(\mathrm{f} \mid u)}\left[f\right] - \mu_u^t\mu_u   -2 y^t \mu_u\right) \\
&= \log \left( \left(2\pi \sigma^{2N}\right)^{-1/2} \right) + \frac{-1}{2\sigma^{2}}\left( y^t y + \mathrm{diag}(\mathrm{Cov}_u)  + \mu_u^t\mu_u   -2 y^t \mu_u\right) \\
&= \log \left( \left(2\pi \sigma^{2N}\right)^{-1/2} \right) + \frac{-1}{2\sigma^{2}}\left\| y - \mu_u\right\|^2 + \frac{-1}{2\sigma^{2}}\left(\mathrm{diag}(\mathrm{Cov}_u) \right) \\
&= \log \left( \left(2\pi \sigma^{2N}\right)^{-1/2} \exp\left(\frac{-1}{2\sigma^{2}}\left\| y - \mu_u\right\|^2\right)\right) + \frac{-1}{2\sigma^{2}}\left(\mathrm{diag}(\mathrm{Cov}_u) \right) \\
&= \log \left(\mathcal{N}\left(y \mid \mu, \sigma^2 I\right)\right) + \frac{-1}{2\sigma^{2}}\left(\mathrm{diag}(\mathrm{Cov}_u) \right) \\
&= \log \left(\frac{-1}{2\sigma^{2}}\left(\mathrm{diag}(\mathrm{Cov}_u) \right)\mathcal{N}\left(y \mid \mu, \sigma^2 I\right)\right)  
\end{aligned}
$$